In [1]:
import pandas as pd
import numpy as np
import math

starting_year_to_filter = 1962
end_year_to_filter = 2020
number_of_lookback_periods = 120


def rank_and_map(df):
    # Make a copy to avoid modifying the original DataFrame
    df_copy = df.copy()
    # Exclude the 'date' column for ranking
    data_columns = df_copy.columns[1:]
    
    # Apply ranking and scaling row-wise (for each date)
    def rank_row(row):
        # Get the ranks (min rank is 1)
        ranks = row.rank(method='min')
        # Normalize the ranks to range between 0 and 1
        ranks_normalized = (ranks - 1) / (len(row) - 1)
        # Map to range [-0.5, 0.5]
        return ranks_normalized - 0.5
    
    # Apply rank_row function to each row, excluding the 'date' column
    df_copy[data_columns] = df_copy[data_columns].apply(rank_row, axis=1)
    return df_copy



def cross_sectional_demean(df):
    # Make a copy to avoid modifying the original DataFrame
    df_copy = df.copy()
    # Exclude the 'date' column
    data_columns = df_copy.columns[1:]
    
    # Apply demeaning row-wise (for each date)
    def demean_row(row):
        row_mean = row.mean()  # Compute the mean of the row
        return row - row_mean  # Subtract the mean from each element in the row
    
    # Apply demean_row function to each row, excluding the 'date' column
    df_copy[data_columns] = df_copy[data_columns].apply(demean_row, axis=1)
    return df_copy


def compute_rs_product(df1, df2):
    # Ensure the date columns match
    if not df1['date'].equals(df2['date']):
        raise ValueError("Date columns of both dataframes must match.")
    
  # Convert to numeric, set invalid values as NaN
    df1 = df1.astype({col: 'float64' for col in df1.columns if col != 'date'})
    df2 = df2.astype({col: 'float64' for col in df2.columns if col != 'date'})
    result = {}
    
    # Iterate over each row (each date)
    for index, date in enumerate(df1['date']):
        # Get the R vector (from df1) and S' vector (from df2) for the current date
        R = df1.iloc[index, 1:].values.reshape(-1, 1)  # n x 1 vector
        S_transpose = df2.iloc[index, 1:].values.reshape(1, -1)  # 1 x n vector
        # Compute the outer product (RS')
        matrix_rs = np.dot(R, S_transpose)  # n x n matrix
        # Store the result in a dictionary, with date as the key
        result[date] = matrix_rs

    return result


def get_prediction_matrix(input_date, result_matrices, n_periods):
    # Sort the dates in result_matrices to ensure they're in order
    sorted_dates = sorted(result_matrices.keys())
    # Find the index of the input date in the sorted list of dates
    if input_date not in sorted_dates:
        raise ValueError("The input date is not found in the result_matrices.")
    
    input_date_index = sorted_dates.index(input_date)
    # Select the last n_periods (excluding the input date)
    start_index = max(0, input_date_index - n_periods)  # Ensure we don't go below index 0
    selected_dates = sorted_dates[start_index:input_date_index]
    
    if len(selected_dates) == 0:
        raise ValueError(f"There are no previous periods to calculate the average for the given number: {n_periods}.")
    
    # Initialize a matrix to accumulate the sum
    matrix_shape = result_matrices[sorted_dates[0]].shape
    sum_matrix = np.zeros(matrix_shape, dtype=float)
    # Sum all the selected matrices
    for date in selected_dates:
        sum_matrix += np.array(result_matrices[date], dtype=float)
    
    # Calculate the element-wise average
    average_matrix = sum_matrix / len(selected_dates)
    return average_matrix



# i should start from 0. In other words, to get the first PP's expected return you must set i=0.
def get_ith_PPs_expected_return(S,i):
    return S[i]

# i should start from 0. In other words, to get the first PP you must set i=0.
def get_ith_position_matrix(U,VT,i):
    u_column = U[:, i]
    v_column = VT[i, :]
    return np.outer(v_column,u_column)

def first_n_PPs_expected_return(S,n):
    sum = 0
    for i in range(n):
        sum += get_ith_PPs_expected_return(S,i)
    return sum

def first_n_PPs_position_matrix(U,VT,number_of_PPs):
    matrix_shape = U.shape
    sum_matrix = np.zeros(matrix_shape, dtype=float)
    for i in range(number_of_PPs):
        sum_matrix += get_ith_position_matrix(U,VT,i)
    return sum_matrix/number_of_PPs

# i should start from 0. In other words, to get the first PEP you must set i=0.
def get_ith_PEPs_expected_return(eigenvalues,i):
    return eigenvalues[i]

def get_ith_symmetric_position_matrix(eigenvectors,i):
    w = eigenvectors[:, i]
    return np.outer(w,w)

def first_n_PEPs_expected_return(eigenvalues,n):
    sum = 0
    for i in range(n):
        sum += abs(get_ith_PEPs_expected_return(eigenvalues,i))
    return sum

def first_n_PEPs_position_matrix(eigenvectors,number_of_PEPs):
    matrix_shape = eigenvectors.shape
    sum_matrix = np.zeros(matrix_shape, dtype=float)
    for i in range(number_of_PEPs):
        sum_matrix += get_ith_symmetric_position_matrix(eigenvectors,i)
    return sum_matrix/number_of_PEPs

def calculate_sharpe_ratio(returns):
    # Compute excess returns
    
    # Compute average excess return
    average_return = returns.mean()
    
    # Compute standard deviation of returns
    std_dev_returns = returns.std()
    
    # Compute Sharpe Ratio
    sharpe_ratio = average_return / std_dev_returns
    
    return sharpe_ratio

In [2]:
df_25_ff_size_value_sorted_monthly = pd.read_csv("data/25_Portfolios_5x5_size_value_monthly.csv")
df_25_ff_size_value_sorted_monthly['date'] = pd.to_datetime(df_25_ff_size_value_sorted_monthly['date'], format='%Y%m') + pd.offsets.MonthEnd(1)
df_25_ff_size_value_sorted_monthly.head(5)

,date,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
0,1926-07-31,5.8248,-1.7006,0.4875,-1.4580,2.0534,1.2077,2.4192,0.4926,-2.6049,...,1.5893,1.5278,1.2978,0.2727,2.4678,3.4539,6.0902,2.0266,3.1111,0.5623
1,1926-08-31,-2.0206,-8.0282,1.3796,1.4606,8.3968,2.3618,-1.1849,4.0084,0.5038,...,1.3336,3.8730,2.0021,2.1706,5.3422,1.0124,4.1903,2.0131,5.4849,7.7576
2,1926-09-30,-4.8291,-2.6154,-4.3417,-3.2729,0.8649,-2.6540,-1.2618,1.0829,-3.5480,...,1.0923,-0.5250,-1.7636,1.4646,0.8730,-1.2906,3.6538,0.0950,-0.7487,-2.4284
3,1926-10-31,-9.3729,-3.5519,-3.4948,3.4413,-2.5476,-2.8069,-3.2663,-5.0745,-8.0191,...,-3.3361,-2.6559,-2.1070,-3.1051,-5.3525,-2.7413,-3.0071,-2.2437,-4.6719,-5.8129
4,1926-11-30,5.5888,4.1877,2.4623,-4.4494,0.5362,3.1033,-2.3690,3.0078,5.1546,...,3.4448,2.3887,3.7335,4.9320,1.8213,4.2946,2.5326,1.5204,3.6619,2.5636


Note that I shift signals one period forward to make computations easier. 

In [3]:
signal_df = pd.DataFrame()
signal_df["date"] = df_25_ff_size_value_sorted_monthly["date"]
signal_df= signal_df.join(df_25_ff_size_value_sorted_monthly.iloc[:, 1:].shift(1))
signal_df.head()

,date,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
0,1926-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1926-08-31,5.8248,-1.7006,0.4875,-1.4580,2.0534,1.2077,2.4192,0.4926,-2.6049,...,1.5893,1.5278,1.2978,0.2727,2.4678,3.4539,6.0902,2.0266,3.1111,0.5623
2,1926-09-30,-2.0206,-8.0282,1.3796,1.4606,8.3968,2.3618,-1.1849,4.0084,0.5038,...,1.3336,3.8730,2.0021,2.1706,5.3422,1.0124,4.1903,2.0131,5.4849,7.7576
3,1926-10-31,-4.8291,-2.6154,-4.3417,-3.2729,0.8649,-2.6540,-1.2618,1.0829,-3.5480,...,1.0923,-0.5250,-1.7636,1.4646,0.8730,-1.2906,3.6538,0.0950,-0.7487,-2.4284
4,1926-11-30,-9.3729,-3.5519,-3.4948,3.4413,-2.5476,-2.8069,-3.2663,-5.0745,-8.0191,...,-3.3361,-2.6559,-2.1070,-3.1051,-5.3525,-2.7413,-3.0071,-2.2437,-4.6719,-5.8129


I can think of this matrix as $S_{t-1}$.

In [4]:
normalized_signal_df = rank_and_map(signal_df)
normalized_signal_df = normalized_signal_df[(normalized_signal_df['date'].dt.year > starting_year_to_filter) & (normalized_signal_df['date'].dt.year < end_year_to_filter)].reset_index(drop=True)
normalized_signal_df.head(5)

,date,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
0,1963-01-31,-0.375000,-0.500000,-0.083333,-0.291667,-0.250000,-0.333333,0.041667,0.000000,-0.166667,...,0.166667,0.416667,0.250000,0.083333,0.375000,0.333333,0.458333,0.500000,0.125000,0.208333
1,1963-02-28,0.500000,0.458333,0.291667,0.333333,0.416667,0.083333,-0.250000,0.041667,0.125000,...,-0.291667,-0.375000,-0.166667,0.250000,-0.083333,-0.333333,-0.458333,-0.500000,0.000000,-0.041667
2,1963-03-31,-0.333333,-0.416667,0.250000,0.333333,0.500000,-0.500000,-0.125000,-0.208333,-0.083333,...,-0.375000,0.083333,0.208333,0.166667,0.375000,-0.291667,-0.041667,-0.166667,0.041667,0.125000
3,1963-04-30,0.416667,-0.500000,-0.458333,-0.333333,0.041667,-0.375000,-0.250000,-0.208333,0.166667,...,0.000000,-0.041667,0.083333,0.208333,0.333333,0.291667,0.125000,0.375000,0.458333,0.500000
4,1963-05-31,0.166667,-0.416667,-0.458333,-0.166667,-0.333333,-0.375000,0.291667,-0.500000,0.083333,...,-0.041667,-0.208333,0.333333,0.458333,0.125000,0.208333,0.041667,0.375000,0.000000,0.500000


This matrix can be denoted as $R_{t-1}$

In [5]:
demeaned_return_df = cross_sectional_demean(df_25_ff_size_value_sorted_monthly)
demeaned_return_df = demeaned_return_df[(demeaned_return_df['date'].dt.year > starting_year_to_filter) & (demeaned_return_df['date'].dt.year < end_year_to_filter)].reset_index(drop=True)
demeaned_return_df.tail()


,date,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
679,2019-08-31,1.380000,-0.234300,-1.924600,-1.220800,-3.292600,4.298100,-1.446800,0.646100,-0.474900,...,3.404900,0.871800,0.542800,0.492400,-2.744100,5.204500,3.154800,2.436200,2.128900,-3.172600
680,2019-09-30,-6.412376,-2.835476,0.537424,2.558724,4.799924,-4.498776,-0.902976,1.052124,2.707224,...,-6.086676,0.171824,2.442424,3.333124,2.160724,-2.471976,-1.386176,0.022424,1.827424,2.442524
681,2019-10-31,-1.093368,-1.462268,-0.764568,-0.603268,-2.230568,2.102732,3.098632,0.030932,1.000732,...,1.115632,-0.155868,-1.944568,-1.678768,-4.873168,1.340632,0.089132,1.139232,-0.020568,2.020132
682,2019-11-30,3.727004,1.997604,-1.184596,-0.627796,-0.099096,3.773904,-0.842596,-1.198496,-0.558996,...,3.398204,-0.277196,-1.237496,-1.185796,-0.394896,-0.337696,0.299304,-1.902396,-1.284096,0.655604
683,2019-12-31,0.075316,8.936616,-0.995984,0.669816,3.443016,0.148416,-1.833284,2.549416,-1.270884,...,-4.280284,-0.877984,-0.037584,-1.262984,2.042116,-0.146884,-2.210784,-1.520384,-0.769384,0.201916


This gives: $R_{t}S'_{t}$

In [6]:
rs_matrix = compute_rs_product(demeaned_return_df, normalized_signal_df)

Prediction matrix for date T+1, used returns data up to month T and signals data up to month T-1. In the function get_prediction_matrix, I start the calculations from the previous month. Note that although the input date is the current data, but the in the function that month is excluded.

Note that in calculating realized returns, I am using the current month(the month of rearlized returns) as index. But remember that the matrix was $S_{t-1}$. So, the index actually retreives the value of the previous month. I formed the matrix this way in order to make the calculations easier.

In [7]:
# I leave out the first 120 observations to compute the prediction matrix.
number_of_PPs_to_consider = 3
number_of_PEPs_to_consider = 3
realized_returns_df = pd.DataFrame(columns=[
    "return_of_simple_factor", 
    "realized_return_of_first_three_PP", 
    "expected_return_of_first_three_PP",
    "realized_return_of_first_three_PEP",
    "expected_return_of_first_three_PEP"
])

for year_month_index in demeaned_return_df.iloc[number_of_lookback_periods:]['date']:
    date_to_consider = pd.Timestamp(year_month_index)
    
    #for PP's
    prediction_matrix = get_prediction_matrix(date_to_consider, rs_matrix, number_of_lookback_periods)
    U, S, VT = np.linalg.svd(prediction_matrix)

    #for PEP's
    Symmetric_prediction_matrix = (prediction_matrix + prediction_matrix.T)/2
    eigenvalues, eigenvectors = np.linalg.eig(Symmetric_prediction_matrix)
    idx = eigenvalues.argsort()[::-1]  # Sort in descending order
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    #to calculate realized returns
    signal_vector = normalized_signal_df[normalized_signal_df.date == date_to_consider].values[0, 1:].reshape(1, -1)  # 1*n matrix
    return_vector = df_25_ff_size_value_sorted_monthly[df_25_ff_size_value_sorted_monthly.date == date_to_consider].values[0, 1:].reshape(-1, 1)  # n*1  # there is not much difference between using demeaned returns or not_demeaned ones. I can replace df_25_ff_size_value_sorted_monthly with demeaned_return_df.
    

    # Compute realized returns
    return_of_simple_factor = (signal_vector @ return_vector)[0][0]
    realized_return_of_first_three_PP = (signal_vector @ first_n_PPs_position_matrix(U, VT, number_of_PPs_to_consider) @ return_vector)[0][0]
    expected_return_of_first_three_PP = first_n_PPs_expected_return(S, number_of_PPs_to_consider)
    realized_return_of_first_three_PEP = (signal_vector @ first_n_PEPs_position_matrix(eigenvectors,number_of_PEPs_to_consider) @ return_vector)[0][0]
    expected_return_of_first_three_PEP = first_n_PEPs_expected_return(eigenvalues, number_of_PEPs_to_consider)

    # Prepare a list for the current row values
    row_values = [
        return_of_simple_factor,  
        realized_return_of_first_three_PP, 
        expected_return_of_first_three_PP,
        realized_return_of_first_three_PEP,
        expected_return_of_first_three_PEP
    ]

    # Iterate over all Principal Portfolios (up to len(S)) and calculate realized/expected returns for each
    for i in range(len(S)):

        # for PP's
        realized_return_ith_PP = (signal_vector @ get_ith_position_matrix(U, VT, i) @ return_vector)[0][0]
        expected_return_ith_PP = get_ith_PPs_expected_return(S, i)

        # for PEP's
        realized_return_ith_PEP = (signal_vector @ get_ith_symmetric_position_matrix(eigenvectors, i) @ return_vector)[0][0]
        expected_return_ith_PEP = get_ith_PEPs_expected_return(eigenvalues, i)

        # Add the values for realized and expected returns of the ith PP to the row
        row_values.append(realized_return_ith_PP)
        row_values.append(expected_return_ith_PP)

        # Add the values for realized and expected returns of the ith PEP to the row
        row_values.append(realized_return_ith_PEP)
        row_values.append(expected_return_ith_PEP)

        # Dynamically add columns if they don't exist. for PP's.
        realized_col_name_pp = f"realized_return_of_{i+1}_PP"
        expected_col_name_pp = f"expected_return_of_{i+1}_PP"

        # Dynamically add columns if they don't exist. for PEP's.
        realized_col_name_pep = f"realized_return_of_{i+1}_PEP"
        expected_col_name_pep = f"expected_return_of_{i+1}_PEP"
        
        # for PP's
        if realized_col_name_pp not in realized_returns_df.columns:
            realized_returns_df[realized_col_name_pp] = None
        if expected_col_name_pp not in realized_returns_df.columns:
            realized_returns_df[expected_col_name_pp] = None

        #for PEP'S
        if realized_col_name_pep not in realized_returns_df.columns:
            realized_returns_df[realized_col_name_pep] = None
        if expected_col_name_pep not in realized_returns_df.columns:
            realized_returns_df[expected_col_name_pep] = None

    # Append the row to the dataframe
    realized_returns_df.loc[len(realized_returns_df)] = row_values


C:\Users\Amin\AppData\Local\Temp\ipykernel_2748\552003013.py:84: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  realized_returns_df[expected_col_name_pep] = None


In [8]:
sharpe_df = realized_returns_df.drop(realized_returns_df.filter(like="expected").columns, axis=1).apply(lambda col: calculate_sharpe_ratio(col)) * math.sqrt(12)

pp_columns = realized_returns_df.filter(like="PP")
pp_realized_mean_df = pp_columns.filter(like="realized").mean(axis=0)
pp_expected_mean_df = pp_columns.filter(like="expected").mean(axis=0)

pep_columns = realized_returns_df.filter(like="PEP")
pep_realized_mean_df = pep_columns.filter(like="realized").mean(axis=0)
pep_expected_mean_df = pep_columns.filter(like="expected").mean(axis=0)

realized_returns_df.to_csv("temp/realized_returns.csv")
sharpe_df.to_csv("temp/sharpe.csv")

pp_columns.to_csv("temp/pp_columns.csv")
pp_realized_mean_df.to_csv("temp/pp_realized_mean_df.csv")
pp_expected_mean_df.to_csv("temp/pp_expected_mean_df.csv")

pep_columns.to_csv("temp/pep_columns.csv")
pep_realized_mean_df.to_csv("temp/pep_realized_mean_df.csv")
pep_expected_mean_df.to_csv("temp/pep_expected_mean_df.csv")

In [51]:
import numpy as np


# Define matrix B = 0.5 * (A - A.T)
assymetric_prediction_matrix = 0.5 * (prediction_matrix - prediction_matrix.T)
transposed_assymetric_prediction_matrix = assymetric_prediction_matrix.T


# Compute eigenvalues and eigenvectors of B
eigenvalues_ta, eigenvectors_ta = np.linalg.eig(transposed_assymetric_prediction_matrix)

# Sort eigenvalues and eigenvectors by the imaginary part (considering sign)
sorted_indices_ta = np.argsort(-eigenvalues_ta.imag)
sorted_eigenvalues_ta = eigenvalues_ta[sorted_indices_ta].imag
sorted_eigenvectors_ta = eigenvectors_ta[:, sorted_indices_ta] * math.sqrt(2) #sqrt(2) is to make the size of the vectors equal to 1.
                                                                        

positive_indices = np.where(sorted_eigenvalues_ta > 0)

# Extract the filtered eigenvalues and corresponding eigenvectors
filtered_eigenvalues = sorted_eigenvalues_ta[positive_indices]
filtered_eigenvectors = sorted_eigenvectors_ta[:, positive_indices].squeeze()


sorted_eigenvectors_B_imaginary_part = filtered_eigenvectors.imag
sorted_eigenvectors_B_real_part = filtered_eigenvectors.real


# Print the results
#print(B,"\n")
print("Sorted Eigenvalues:")
print(filtered_eigenvalues)

print("eigenvalues")
print(sorted_eigenvectors_B)

print("\nCorresponding Eigenvectors:")
print("imaginary:",sorted_eigenvectors_B_imaginary_part,"\n \n","real:",sorted_eigenvectors_B_real_part)




Sorted Eigenvalues:
[0.41407924 0.24017137 0.16040433 0.14373795 0.09498426 0.07774459
 0.06683157 0.04117282 0.03080145 0.02191644 0.00844341 0.00512527]
eigenvalues
[[-6.12206454e-01+0.00000000e+00j -2.99933808e-01+7.02337619e-02j
   3.80676868e-01+1.39642597e-01j -2.93706388e-01+8.55457351e-02j
   7.70225901e-02+1.83131271e-01j -1.07088837e-01+2.15665467e-01j
   1.70473171e-01+4.70289451e-02j  6.57749420e-02+1.53572102e-01j
  -1.57896010e-01-2.97450270e-02j -3.68353007e-02-5.71423630e-02j
   3.67286865e-02+1.50249604e-01j  1.33086014e-01+6.18774469e-02j
   2.82058065e-01+0.00000000e+00j  1.33086014e-01-6.18774469e-02j
   3.67286865e-02-1.50249604e-01j -3.68353007e-02+5.71423630e-02j
  -1.57896010e-01+2.97450270e-02j  6.57749420e-02-1.53572102e-01j
   1.70473171e-01-4.70289451e-02j -1.07088837e-01-2.15665467e-01j
   7.70225901e-02-1.83131271e-01j -2.93706388e-01-8.55457351e-02j
   3.80676868e-01-1.39642597e-01j -2.99933808e-01-7.02337619e-02j
  -6.12206454e-01-0.00000000e+00j]
 [-2.1

In [53]:
filtered_eigenvalues[0]

0.41407923820330267

In [58]:
if sorted_eigenvectors_B_imaginary_part.ndim == 1:
    print(sorted_eigenvectors_B_imaginary_part[0])
else:
    print(sorted_eigenvectors_B_imaginary_part[:,11])



[ 0.06187745  0.18645539  0.         -0.6204251   0.21867346 -0.06485446
 -0.36970146  0.23473508  0.34588373  0.09231629 -0.11977403  0.03896031
  0.18497476 -0.16967532 -0.05025436 -0.1335723  -0.15150979  0.01593827
  0.04920373  0.01436857  0.25524984 -0.09903048  0.05910485 -0.03041354
  0.00810864]


In [56]:
len(filtered_eigenvalues)

12

In [55]:
sorted_eigenvectors_B_imaginary_part.shape

(25, 12)

In [ ]:
sorted_eigenvectors_B_imaginary_part